In [75]:
!pip install numpy moabb

In [2]:
import numpy as np
from moabb.datasets import Cho2017
from moabb.paradigms import LeftRightImagery
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from mne.decoding import CSP
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.pipeline import make_pipeline
from moabb.evaluations import CrossSessionEvaluation
from moabb.paradigms import FilterBankLeftRightImagery, LeftRightImagery
from moabb.pipelines.utils import FilterBank

In [18]:
# 주파수 필터 대역 설정
filters = [[8, 12], [12, 20], [20, 35]]

# FilterBank 패러다임 설정
paradigm = FilterBankLeftRightImagery(filters=filters)

# 데이터 가져오기
dataset = Cho2017()
X, y, metadata = paradigm.get_data(dataset=dataset, subjects=[30])

# 데이터 형태 확인 (X.shape: (n_trials, n_channels, n_times, n_filters))
print("Original X shape:", X.shape)

# 각 필터별로 CSP 적용 및 결과 결합
X_csp_all = []
for i in range(X.shape[-1]):  # 필터별로 반복
    X_filter = X[..., i]  # i번째 필터의 데이터 추출 (3차원: (n_trials, n_channels, n_times))
    csp = CSP(n_components=8)
    X_csp_filter = csp.fit_transform(X_filter, y)  # CSP 적용
    X_csp_all.append(X_csp_filter)

# 필터별 결과를 병합 (예: (n_trials, 24) 형태)
X_csp_combined = np.concatenate(X_csp_all, axis=1)

print("Combined CSP shape:", X_csp_combined.shape)

# 데이터 분리 및 모델 학습
X_train, X_test, y_train, y_test = train_test_split(X_csp_combined, y, test_size=0.3, random_state=42)

# LDA 모델 학습 및 평가
clf = LDA()
clf.fit(X_train, y_train)

# 예측 및 평가
predict = clf.predict(X_test)
accuracy = accuracy_score(y_test, predict)
print(f"FilterBank CSP + LDA 정확도 (Accuracy): {accuracy:.2f}")

# 혼동 행렬과 분류 리포트 출력
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, predict))

print("\nClassification Report:")
print(classification_report(y_test, predict))


C:\Users\kimso\anaconda3\envs\csp_lda\lib\site-packages\urllib3\connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 's3.ap-northeast-1.wasabisys.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
100%|###############################################| 204M/204M [00:00<?, ?B/s]
SHA256 hash of downloaded file: 29d0d8714bb8e59cc9c3718ff84542536f34703eb17f536e7400d39abf5561f8
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.
Trials demeaned and stacked with zero buffer to create continuous data -- edge effects present
C:\Users\kimso\anaconda3\envs\csp_lda\lib\site-packages\moabb\datasets\preprocessing.py:279: UserWarning: warnEpochs <Epochs | 200 events (all good), 0 – 3 s (baseline off), ~150.2 MiB, data loaded,
 'left_hand': 100
 'right_hand': 100>
  wa

Original X shape: (200, 64, 1537, 3)
Computing rank from data with rank=None
    Using tolerance 2.5e+03 (2.2e-16 eps * 64 dim * 1.7e+17  max singular value)
    Estimated rank (data): 64
    data: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating class=left_hand covariance using EMPIRICAL
Done.
Estimating class=right_hand covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 2.7e+03 (2.2e-16 eps * 64 dim * 1.9e+17  max singular value)
    Estimated rank (data): 64
    data: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating class=left_hand covariance using EMPIRICAL
Done.
Estimating class=right_hand covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 3.2e+03 (2.2e-16 eps * 64 dim * 2.2e+17  max singular value)
    Estimated rank (data): 64
    data: rank 64 computed from 64 data channels with 0 projectors
R

In [19]:
accuracy = accuracy_score(y_test, predict) * 100
print(f"FilterBank CSP + LDA 정확도 (Accuracy): {accuracy:.2f}%")

FilterBank CSP + LDA 정확도 (Accuracy): 96.67%
